In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import json
import pickle
import argparse
import torch
import re
from kobert import get_pytorch_kobert_model
from kobert import get_tokenizer
from model.net import KobertSequenceFeatureExtractor, KobertCRF, KobertBiLSTMCRF, KobertBiGRUCRF
from gluonnlp.data import SentencepieceTokenizer
from data_utils.utils import Config
from data_utils.vocab_tokenizer import Tokenizer
from data_utils.pad_sequence import keras_pad_fn
from pathlib import Path

import numpy as np
import itertools

from konlpy.tag import Hannanum
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from krwordrank.word import summarize_with_keywords

In [2]:
hannanum = Hannanum()
#한국어 불용어
stop_words="아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓"
stop_words=stop_words.split(' ')

# SBERT를 로드
sbert_model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [3]:
class DecoderFromNamedEntitySequence():
    def __init__(self, tokenizer, index_to_ner):
        self.tokenizer = tokenizer
        self.index_to_ner = index_to_ner

    def __call__(self, list_of_input_ids, list_of_pred_ids):
        input_token = self.tokenizer.decode_token_ids(list_of_input_ids)[0]
        pred_ner_tag = [self.index_to_ner[pred_id] for pred_id in list_of_pred_ids[0]]

        print("len: {}, input_token:{}".format(len(input_token), input_token))
        print("len: {}, pred_ner_tag:{}".format(len(pred_ner_tag), pred_ner_tag))

        # ----------------------------- parsing list_of_ner_word ----------------------------- #
        list_of_ner_word = []
        entity_word, entity_tag, prev_entity_tag = "", "", ""
        for i, pred_ner_tag_str in enumerate(pred_ner_tag):
            if "B-" in pred_ner_tag_str:
                entity_tag = pred_ner_tag_str[-3:]

                if prev_entity_tag != entity_tag and prev_entity_tag != "":
                    list_of_ner_word.append({"word": entity_word.replace("▁", " "), "tag": prev_entity_tag, "prob": None})

                entity_word = input_token[i]
                prev_entity_tag = entity_tag
            elif "I-"+entity_tag in pred_ner_tag_str:
                entity_word += input_token[i]
            else:
                if entity_word != "" and entity_tag != "":
                    list_of_ner_word.append({"word":entity_word.replace("▁", " "), "tag":entity_tag, "prob":None})
                entity_word, entity_tag, prev_entity_tag = "", "", ""


        # ----------------------------- parsing decoding_ner_sentence ----------------------------- #
        decoding_ner_sentence = ""
        is_prev_entity = False
        prev_entity_tag = ""
        is_there_B_before_I = False

        for token_str, pred_ner_tag_str in zip(input_token, pred_ner_tag):
            token_str = token_str.replace('▁', ' ')  # '▁' 토큰을 띄어쓰기로 교체

            if 'B-' in pred_ner_tag_str:
                if is_prev_entity is True:
                    decoding_ner_sentence += ':' + prev_entity_tag+ '>'

                if token_str[0] == ' ':
                    token_str = list(token_str)
                    token_str[0] = ' <'
                    token_str = ''.join(token_str)
                    decoding_ner_sentence += token_str
                else:
                    decoding_ner_sentence += '<' + token_str
                is_prev_entity = True
                prev_entity_tag = pred_ner_tag_str[-3:] # 첫번째 예측을 기준으로 하겠음
                is_there_B_before_I = True

            elif 'I-' in pred_ner_tag_str:
                decoding_ner_sentence += token_str

                if is_there_B_before_I is True: # I가 나오기전에 B가 있어야하도록 체크
                    is_prev_entity = True
            else:
                if is_prev_entity is True:
                    decoding_ner_sentence += ':' + prev_entity_tag+ '>' + token_str
                    is_prev_entity = False
                    is_there_B_before_I = False
                else:
                    decoding_ner_sentence += token_str

        return list_of_ner_word, decoding_ner_sentence

In [4]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [5]:
#parser, ner_model경로 설정

parser = argparse.ArgumentParser()
parser.add_argument('--data_dir', default='./data_in', help="Directory containing config.json of data")
parser.add_argument('--model_dir', default='./experiments/base_model_with_crf_val', help="Directory containing config.json of model")
args, _ = parser.parse_known_args()
ner_model_dir = Path(args.model_dir)
ner_model_config = Config(json_path=ner_model_dir / 'config.json')

In [6]:
# Vocab & Tokenizer
# tok_path = get_tokenizer() # ./tokenizer_78b3253a26.model
tok_path = "./ptr_lm_model/tokenizer_78b3253a26.model"
ptr_tokenizer = SentencepieceTokenizer(tok_path)

# load vocab & tokenizer
with open(ner_model_dir / "vocab.pkl", 'rb') as f:
    vocab = pickle.load(f)

tokenizer = Tokenizer(vocab=vocab, split_fn=ptr_tokenizer, pad_fn=keras_pad_fn, maxlen=ner_model_config.maxlen)

# load ner_to_index.json
with open(ner_model_dir / "ner_to_index.json", 'rb') as f:
    ner_to_index = json.load(f)
    index_to_ner = {v: k for k, v in ner_to_index.items()}

ner_model = KobertCRF(config=ner_model_config, num_classes=len(ner_to_index), vocab=vocab)

# load
ner_model_dict = ner_model.state_dict()
checkpoint = torch.load("./experiments/base_model_with_crf_val/best-epoch-9-step-750-acc-0.957.bin", map_location=torch.device('cpu'))

convert_keys = {}
for k, v in checkpoint['model_state_dict'].items():
    new_key_name = k.replace("module.", '')
    if new_key_name not in ner_model_dict:
        print("{} is not int model_dict".format(new_key_name))
        continue
    convert_keys[new_key_name] = v

ner_model.load_state_dict(convert_keys)
ner_model.eval()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# n_gpu = torch.cuda.device_count()
# if n_gpu > 1:
#     model = torch.nn.DataParallel(model)
ner_model.to(device)

decoder_from_res = DecoderFromNamedEntitySequence(tokenizer=tokenizer, index_to_ner=index_to_ner)

In [7]:
doc_with_newlines = """
[IT동아 남시현 기자] 지난 3월 31일, 인텔이 아크(Arc) 그래픽 카드 라인업을 공개하며 그래픽 카드 사업 진출을 선언했다. 2018년 6월에 외장 그래픽 카드 개발 여부를 공개한 지 4년 만의 결과다. 인텔의 외장 그래픽 카드는 인텔 Xe 마이크로 아키텍처를 기반으로 하며, 노트북용 그래픽 카드를 시작으로 올해 여름까지 데스크톱 제품까지 차례로 공개한다. 이로써 지난 20년 간 엔비디아와 AMD가 나눠갖던 그래픽 카드 시장은 이제 세 개 기업의 경쟁 구도로 경쟁하게 된다.가장 처음 공개된 제품은 삼성전자 갤럭시 북2 프로에 탑재된 인텔 아크 A350M이다. A350M은 전체 노트북 그래픽 카드 라인업 중 가장 보급형인 제품이며, A350M을 시작으로 A370M, A550M, A730M, A770M의 순서로 성능이 향상된다. A 시리즈는 기존 GPU 벡터 장치 대비 16배가 향상된 인공지능 가속 기능 MXM AI 엔진과 최대 8K60p 12bit HDR 디코딩과 8K 10bit HDR 인코딩을 지원하는 Xe 미디어 엔진, 최대 두 대의 8K60p HDR 및 네 대의 4K 120p HDR 영상을 출력하는 Xe HPG 디스플레이 엔진을 탑재한다. 게이밍 역시 인공지능 기반의 해상도 향상 기능인 XeSS, 실시간 광선 추적 등이 적용된다. 갤럭시 북2 프로를 토대로 자세한 성능을 알아보자.인텔 아크 A350M은 인텔 아크 A 시리즈 그래픽 카드 중 가장 보급형 그래픽카드로, 6개의 Xe 코어와 6개의 레이 트레이싱(실시간 광선 추적) 유닛, 4GB의 GDDR6 메모리가 적용된다. 노트북용 그래픽 카드 중 가장 최상급인 A770M은 32개의 Xe 코어 및 레이 트레이싱 유닛, 16GB의 GDDR6 메모리를 갖추고 있다. 그래픽 카드의 전력은 25~35W 수준으로 게이밍 노트북보다는 고성능 경량 노트북에 적합하지만, H265와 AV1, VP9에 대한 하드웨어 인코딩 및 디코딩을 지원해 가벼운 게임 혹은 영상 편집 용도로 적합하다.컴퓨터의 프로세서 및 그래픽 칩셋 스펙을 확인하는 CPU-Z(좌측) 및 GPU-Z(우측)를 활용해 삼성전자 갤럭시 북2 프로의 스펙을 확인했다. 해당 노트북은 28W 프로세서인 12세대 인텔 코어 i5-1240P를 탑재하며, 그래픽 카드는 인텔 아크 A350M으로 확인된다. 그래픽 카드의 연산 성능을 짐작할 수 있는 픽셀 필레이트는 초당 70.4기가 픽셀, 3D화 하는 텍스처 필레이트는 초당 140.8기가 텍셀로 엔비디아 지포스 GTX 1660 Ti 모바일과 거의 비슷하다.그래픽 드라이버는 ‘인텔 아크 컨트롤’이라는 독자 소프트웨어로 구동된다. 그래픽 드라이버는 그래픽 카드의 기능 및 성능 관리, 게임 최적화 및 호환성 등을 위한 소프트웨어로, 작성일 기준 최신 버전은 30.0.101.1330 버전이다. 아크 컨트롤에서는 그래픽 드라이버의 최신 기능에 대한 소개와 지원 게임 연결, 그래픽 카드 온도 및 상황 등을 확인할 수 있는 퍼포먼스, 그리고 라이브 스트리밍 보조 기능인 스튜디오 기능이 제공된다. 자주 사용하는 기능은 위젯으로 정렬할 수 있고, 설정을 통해 서비스 설정 등을 바꿀 수 있다. 특히 엔비디아 지포스 익스피리언스와 달리 로그인하지 않고도 기능이 제공된다.생산성과 게이밍 성능은 어떤 수준? 인텔 아크 A350M의 성능을 확인해보고자 생산성 성능을 점수로 환산하는 프로그램, PC마크10 벤치마크를 활용해 성능을 측정해봤다. PC마크10은 앱 실행 속도나 화상 회의, 웹 브라우징, 액셀 및 워드 문서 편집, 사진 및 영상 편집, 3D 렌더링 등의 작업을 수행해 CPU 및 그래픽 카드의 작업 성능을 점수로 평가한다. PC마크 10을 활용해 확인한 인텔 i5-1240P 및 인텔 아크 A350M의 종합 점수는 5천355점, 그래픽 카드의 영향을 받는 디지털 콘텐츠 생산성 점수는 6천813점으로 확인된다. 엔비디아 지포스 MX450보다는 1천800점가량 높고, AMD 라데온 RX 5300M과 비슷한 점수다. 보급형 라인업인 만큼 RTX 3050보다는 낮지만, 초경량 노트북에 탑재된 그래픽 카드라는 점을 감안하면 좋은 점수다.인텔 아크 시리즈의 또 다른 특징은 영상 편집 성능이다. 인텔 아크의 Xe 미디어 엔진은 8K HDR 인코딩 및 디코딩을 포함해 H265, AV1, VP9 등 최신 코덱을 대거 지원한다. 또한 인텔 CPU의 내장 그래픽과 아크 그래픽 카드가 동시에 동작하는 하이퍼 인코딩 기능을 활용해 작업 효율을 끌어올리고 있다. 동영상 변환 프로그램인 ‘핸드브레이크’를 활용해 2분 13초 길이의 H.264 4K 30프레임, 비트레이트 60Mbps의 영상을 동일 크기의 H.265 영상으로 변환했다. 인텔 i5-1240P의 CPU만 활용한 편집에 걸린 시간은 10분 40초인데, 인텔 아이리스 Xe 내장 그래픽과 인텔 아크 A350M이 하이퍼 인코딩으로 동작했을 때 걸린 시간은 44초에 불과했다. 영상 편집 효율은 동급 구성의 타사 제품보다 좋을 것으로 판단된다.게이밍 성능을 확인하기에 앞서, 게이밍 성능을 점수로 환산하는 프로그램인 3D 마크의 타임스파이, 파이어 스트라이크를 각각 실행해 점수를 확인했다. 아이리스 Xe 내장 그래픽의 점수는 1천352점일 때, 인텔 아크 A350M의 점수는 2천976점으로 확인된다. 엔비디아 지포스 MX450의 타임스파이 점수가 1천900점 대, 지포스 GTX 1650 모바일의 타임스파이 점수가 3천500대이므로, 경량 노트북으로는 높은 편이다. 다이렉트X 11 기반 테스트인 파이어 스트라이크 점수는 7천287점으로, GTX1650 모바일의 8천 점과 비슷한 수준이다. 게이밍 성능은 라이즈 오브 더 툼레이더와 토탈 워 사가:트로이, 토탈워:워해머 II, 레드 데드 리뎀션 II, GTA V까지 다양하게 수행했다. 라이즈 오브 더 툼레이더는 2016년 출시된 게임으로, 권장 사양은 지포스 GTX970과 AMD 라데온 R9 290X다. 테스트는 FHD 해상도, 중간 옵션으로 진행했다. 이때 종합 점수는 41.04FPS로 약간은 끊어진다고 느낄 수준이다. 다행히 30프레임 이상은 확보하므로 게임 자체를 즐기는 데는 무리가 없으며, 사양을 낮추면 60프레임도 가능한 정도다. CPU 성능도 함께 중요한 토탈 워 사가:트로이와 토탈워:워해머 II를 각각 실행해봤다. 두 게임 모두 GTX 970 혹은 R9 290X 정도는 사용해야 한다. 먼저 토탈 워 사가: 트로이는 ‘높음’ 성능을 설정한 다음 배틀 벤치마크를 수행했다. 이때 인텔 아크 A350M의 프레임은 27.7fps로 무난한 수준이다. 반면 토탈워:워해머 II는 높음 설정에서 22.8fps로 다소 사양을 낮출 필요가 있다. 두 게임 모두 프로세서의 영향을 많이 받는 게임이지만, 옵션 타협으로 충분히 게임을 즐길 수준으로 볼 수 있다. 이보다 높은 사양의 CPU와 GPU를 조합하면 토탈워:삼국이나 토탈워:워해머 III는 무난하게 즐길 수 있을 것으로 보인다. 레드 데드 리뎀션 II는 720p 중간 옵션으로 설정해야 무난하게 플레이할 수 있다. 마지막으로 오픈월드 게임인 레드 데드 리뎀션II와 GTA V를 각각 실행했다. 레드 데드 리뎀션 II는 권장 사양이 GTX 1060 6GB, 라데온 RX 480 4G로 높은 게임이다. 그렇다 보니 FHD 해상도 보통 사양에서도 평균 22프레임으로 나타났다. 이 정도 사양의 게임을 플레이한다면 HD급 해상도로 조정해야 한다. 반면 권장 사양이 GTX 660 2GB 및 라데온 HD 7870으로 낮은 GTA V는 FHD 해상도 벤치마크에서 평균 77.1프레임, 최소 27.5프레임으로 안정적인 모습을 보여주었다. 배틀그라운드나 디아블로 2: 레저렉션 등의 게임도 중간 수준의 옵션이면 60프레임 내외로 즐길 수 있을 정도다. 인텔 아크 그래픽 카드는 인텔이 20여 년 만에 다시 공개한 그래픽 카드다. 완전히 새롭게 설계된 만큼 최신 코덱 지원과 세련된 공정으로 제작됐고, 인텔 아크 A350M으로만 확인한 정보만으로도 엔비디아는 어려워도 AMD는 긴장해야 할 수준이다. 다만, 그래픽 카드 생태계가 이제 막 시작한 터라 호환성이 부족한 점은 어쩔 수 없다. 테스트를 진행하는 과정에서도 3D 렌더링 벤치마크인 블랜더, V-레이가 그래픽 카드를 인식하지 못한다거나, 썬더볼트용 외장 디스플레이가 제대로 연결되지 않는 등의 문제가 있었다. 아크 컨트롤의 세부 메뉴도 로딩을 무한 반복한다거나, 공란으로 남아있는 등 미완성된 모습을 보여주었다. 드라이버가 업데이트될 수록 문제가 개선되겠지만, 아직은 갈 길이 멀어 보인다. 성능은 기대했던 수준이다. 가장 낮은 라인업의 제품임에도 엔비디아의 MX450M보다는 낫고, 생산성은 GTX 1650 Ti에 근접한 수준이다. 특히 GTX 1650 Ti는 별도로 방열 설계가 필요한 게이밍 노트북인데, 훨씬 더 가볍고 방열 성능이 적은 경량 노트북으로 따라잡았다는 점이 인상적이다. 추후 드라이버가 안정화된다면 훨씬 나은 수준을 보여줄 것으로 보인다. 인텔 아크 시리즈는 확실히 잘 만든 그래픽 카드다. 다듬어야 할 부분이 많지만, 엔비디아와 AMD 모두 간만에 등장한 경쟁에 긴장의 끈을 붙들어 매야 할 것으로 보인다.
"""
doc=doc_with_newlines.replace(".", "\n")
#한글, 영어, 숫자, +, -, 개행, 띄어쓰기만 놔두고 다 제거
#doc = re.sub(r"[^가-힣a-zA-Z0-9\+\-\n\s]","",doc)

#문장을 분리하였지만, 아직 앞뒤 공백이 있음
temp_texts =  doc.split('\n')
#앞뒤 공백이 제거된 문자열들이 저장되는 list
texts=[]
for text in temp_texts:
    text = text.strip()
    if text is not '':
        texts.append(text)

print(texts)

['[IT동아 남시현 기자] 지난 3월 31일, 인텔이 아크(Arc) 그래픽 카드 라인업을 공개하며 그래픽 카드 사업 진출을 선언했다', '2018년 6월에 외장 그래픽 카드 개발 여부를 공개한 지 4년 만의 결과다', '인텔의 외장 그래픽 카드는 인텔 Xe 마이크로 아키텍처를 기반으로 하며, 노트북용 그래픽 카드를 시작으로 올해 여름까지 데스크톱 제품까지 차례로 공개한다', '이로써 지난 20년 간 엔비디아와 AMD가 나눠갖던 그래픽 카드 시장은 이제 세 개 기업의 경쟁 구도로 경쟁하게 된다', '가장 처음 공개된 제품은 삼성전자 갤럭시 북2 프로에 탑재된 인텔 아크 A350M이다', 'A350M은 전체 노트북 그래픽 카드 라인업 중 가장 보급형인 제품이며, A350M을 시작으로 A370M, A550M, A730M, A770M의 순서로 성능이 향상된다', 'A 시리즈는 기존 GPU 벡터 장치 대비 16배가 향상된 인공지능 가속 기능 MXM AI 엔진과 최대 8K60p 12bit HDR 디코딩과 8K 10bit HDR 인코딩을 지원하는 Xe 미디어 엔진, 최대 두 대의 8K60p HDR 및 네 대의 4K 120p HDR 영상을 출력하는 Xe HPG 디스플레이 엔진을 탑재한다', '게이밍 역시 인공지능 기반의 해상도 향상 기능인 XeSS, 실시간 광선 추적 등이 적용된다', '갤럭시 북2 프로를 토대로 자세한 성능을 알아보자', '인텔 아크 A350M은 인텔 아크 A 시리즈 그래픽 카드 중 가장 보급형 그래픽카드로, 6개의 Xe 코어와 6개의 레이 트레이싱(실시간 광선 추적) 유닛, 4GB의 GDDR6 메모리가 적용된다', '노트북용 그래픽 카드 중 가장 최상급인 A770M은 32개의 Xe 코어 및 레이 트레이싱 유닛, 16GB의 GDDR6 메모리를 갖추고 있다', '그래픽 카드의 전력은 25~35W 수준으로 게이밍 노트북보다는 고성능 경량 노트북에 적합하지만, H265와 AV1, VP9에 대한 하드웨어 인코딩 및 디코딩을 지원해 가벼운 게임 혹은 영상 편집 용

In [8]:
# 품사 태깅
tokenized_doc = hannanum.pos(doc)
#print(tokenized_doc)
# 형태소 분석기를 통해 명사 추출한 문서를 만듦
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc
                   if (word[1] == 'N' or word[1] == 'F')
                   and not word[0].isdigit() 
                   and len(word[0])>1
                   and word[0] not in stop_words
                   and word[0].isalnum()])

n_gram_range = (1, 1)
# CountVectorizer를 사용하여 n-gram을 추출(n_gram_range에 해당하는)
count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names()

#doc를 임베딩
doc_embedding = sbert_model.encode([doc])
#text들을 임베딩
texts_embedding = sbert_model.encode(texts)
#n_gram들을 임베딩
candidate_embeddings = sbert_model.encode(candidates)

In [9]:
org = set([])
per = set([])
poh = set([])

for text in texts:
    list_of_input_ids = tokenizer.list_of_string_to_list_of_cls_sep_token_ids([text])
    ner_model_input = torch.tensor(list_of_input_ids).long()

    ## for bert alone
    # y_pred = model(x_input)
    # list_of_pred_ids = y_pred.max(dim=-1)[1].tolist()

    ## for bert crf
    list_of_pred_ids = ner_model(ner_model_input)

    list_of_ner_word, decoding_ner_sentence = decoder_from_res(list_of_input_ids=list_of_input_ids, list_of_pred_ids=list_of_pred_ids)
    print("decoding_ner_sentence:", decoding_ner_sentence)
    print("list_of_ner_word", list_of_ner_word)
    
    for ner_word in list_of_ner_word:
        tag=ner_word['tag']
        word = ner_word['word'].strip()
        if(tag == 'ORG' and len(word)>1):
            org.add(word)
        if(tag == 'PER' and len(word)>1):
            per.add(word)
        if(tag == 'POH' and len(word)>1):
            poh.add(word)

/Users/baeseonghyun/Desktop/keyword/pytorch-crf/torchcrf/__init__.py:305: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_croot-udngs7fm/pytorch_1648016055234/work/aten/src/ATen/native/TensorCompare.cpp:328.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


len: 41, input_token:['[CLS]', '▁[', 'IT', '동', '아', '▁남', '시', '현', '▁', '기자', ']', '▁지난', '▁3', '월', '▁31', '일', ',', '▁인', '텔', '이', '▁아', '크', '(', 'A', 'r', 'c', ')', '▁그래픽', '▁카드', '▁라인업', '을', '▁공개', '하며', '▁그래픽', '▁카드', '▁사업', '▁진출', '을', '▁선언', '했다', '[SEP]']
len: 41, pred_ner_tag:['[CLS]', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'B-DAT', 'I-DAT', 'I-DAT', 'I-DAT', 'O', 'B-ORG', 'I-ORG', 'O', 'B-POH', 'I-POH', 'O', 'O', 'I-POH', 'I-POH', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '[SEP]']
decoding_ner_sentence: [CLS] <[IT동아:ORG> <남시현:PER> 기자] 지난 <3월 31일:DAT>, <인텔:ORG>이 <아크:POH>(Arc) 그래픽 카드 라인업을 공개하며 그래픽 카드 사업 진출을 선언했다[SEP]
list_of_ner_word [{'word': ' [IT동아', 'tag': 'ORG', 'prob': None}, {'word': ' 남시현', 'tag': 'PER', 'prob': None}, {'word': ' 3월 31일', 'tag': 'DAT', 'prob': None}, {'word': ' 인텔', 'tag': 'ORG', 'prob': None}, {'word': ' 아크', 'tag': 'POH', 'prob': None}]
len: 22, input_token:['[CLS]', '▁20', '

len: 49, input_token:['[CLS]', '▁컴퓨터', '의', '▁프로', '세', '서', '▁및', '▁그래픽', '▁', '칩', '셋', '▁스', '펙', '을', '▁확인', '하는', '▁C', 'P', 'U', '-', 'Z', '(', '좌', '측', ')', '▁및', '▁G', 'P', 'U', '-', 'Z', '(', '우', '측', ')', '를', '▁활용해', '▁삼성전자', '▁갤럭시', '▁북', '2', '▁프로', '의', '▁스', '펙', '을', '▁확인', '했다', '[SEP]']
len: 49, pred_ner_tag:['[CLS]', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'B-POH', 'I-POH', 'I-POH', 'I-POH', 'O', 'O', 'O', 'O', 'O', 'O', '[SEP]']
decoding_ner_sentence: [CLS] 컴퓨터의 프로세서 및 그래픽 칩셋 스펙을 확인하는 CPU-Z(좌측) 및 GPU-Z(우측)를 활용해 <삼성전자:ORG> <갤럭시 북2 프로:POH>의 스펙을 확인했다[SEP]
list_of_ner_word [{'word': ' 삼성전자', 'tag': 'ORG', 'prob': None}, {'word': ' 갤럭시 북2 프로', 'tag': 'POH', 'prob': None}]
len: 42, input_token:['[CLS]', '▁해당', '▁노트북', '은', '▁28', 'W', '▁프로', '세', '서', '인', '▁12', '세대', '▁인', '텔', '▁코', '어', '▁', 'i', '5', '-', '12', '40', '

len: 31, input_token:['[CLS]', '▁보급', '형', '▁라인업', '인', '▁만큼', '▁R', 'T', 'X', '▁30', '50', '보다는', '▁낮', '지만', ',', '▁초', '경', '량', '▁노트북', '에', '▁탑재', '된', '▁그래픽', '▁카드', '라는', '▁점을', '▁감안하면', '▁좋은', '▁점수', '다', '[SEP]']
len: 31, pred_ner_tag:['[CLS]', 'O', 'O', 'O', 'O', 'O', 'B-POH', 'I-POH', 'I-POH', 'I-POH', 'I-POH', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '[SEP]']
decoding_ner_sentence: [CLS] 보급형 라인업인 만큼 <RTX 3050:POH>보다는 낮지만, 초경량 노트북에 탑재된 그래픽 카드라는 점을 감안하면 좋은 점수다[SEP]
list_of_ner_word [{'word': ' RTX 3050', 'tag': 'POH', 'prob': None}]
len: 16, input_token:['[CLS]', '▁인', '텔', '▁아', '크', '▁시리즈', '의', '▁또', '▁다른', '▁특징', '은', '▁영상', '▁편집', '▁성능', '이다', '[SEP]']
len: 16, pred_ner_tag:['[CLS]', 'B-POH', 'I-POH', 'I-POH', 'I-POH', 'I-POH', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '[SEP]']
decoding_ner_sentence: [CLS] <인텔 아크 시리즈:POH>의 또 다른 특징은 영상 편집 성능이다[SEP]
list_of_ner_word [{'word': ' 인텔 아크 시리즈', 'tag': 'POH', 'prob': None}

len: 15, input_token:['[CLS]', '▁테스트', '는', '▁F', 'HD', '▁해상', '도', ',', '▁중간', '▁', '옵션', '으로', '▁진행', '했다', '[SEP]']
len: 15, pred_ner_tag:['[CLS]', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '[SEP]']
decoding_ner_sentence: [CLS] 테스트는 FHD 해상도, 중간 옵션으로 진행했다[SEP]
list_of_ner_word []
len: 9, input_token:['[CLS]', '▁이', '때', '▁종합', '▁점수', '는', '▁', '41', '[SEP]']
len: 9, pred_ner_tag:['[CLS]', 'O', 'O', 'O', 'O', 'O', 'B-NOH', 'I-NOH', '[SEP]']
decoding_ner_sentence: [CLS] 이때 종합 점수는 <41:NOH>[SEP]
list_of_ner_word [{'word': ' 41', 'tag': 'NOH', 'prob': None}]
len: 17, input_token:['[CLS]', '▁', '04', 'F', 'PS', '로', '▁약', '간', '은', '▁', '끊', '어', '진다', '고', '▁느낄', '▁수준이다', '[SEP]']
len: 17, pred_ner_tag:['[CLS]', 'B-NOH', 'I-NOH', 'I-NOH', 'I-NOH', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '[SEP]']
decoding_ner_sentence: [CLS] <04FPS:NOH>로 약간은 끊어진다고 느낄 수준이다[SEP]
list_of_ner_word [{'word': ' 04FPS', 'tag': 'NOH', 'prob': None}]
len: 40, input_token:['[CLS

len: 21, input_token:['[CLS]', '▁그', '렇', '다', '▁보니', '▁F', 'HD', '▁해상', '도', '▁보통', '▁사', '양', '에서도', '▁평균', '▁22', '프', '레', '임', '으로', '▁나타났다', '[SEP]']
len: 21, pred_ner_tag:['[CLS]', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NOH', 'I-NOH', 'I-NOH', 'I-NOH', 'O', 'O', '[SEP]']
decoding_ner_sentence: [CLS] 그렇다 보니 FHD 해상도 보통 사양에서도 평균 <22프레임:NOH>으로 나타났다[SEP]
list_of_ner_word [{'word': ' 22프레임', 'tag': 'NOH', 'prob': None}]
len: 19, input_token:['[CLS]', '▁이', '▁정도', '▁사', '양', '의', '▁게임', '을', '▁플레이', '한다면', '▁', 'HD', '급', '▁해상', '도로', '▁조정', '해야', '▁한다', '[SEP]']
len: 19, pred_ner_tag:['[CLS]', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NOH', 'I-NOH', 'O', 'O', 'O', 'O', 'O', 'O', '[SEP]']
decoding_ner_sentence: [CLS] 이 정도 사양의 게임을 플레이한다면 <HD:NOH>급 해상도로 조정해야 한다[SEP]
list_of_ner_word [{'word': ' HD', 'tag': 'NOH', 'prob': None}]
len: 43, input_token:['[CLS]', '▁반면', '▁권', '장', '▁사', '양', '이', '▁G', 'T', 'X', '▁6', '60', '▁2', 'GB', '▁및', '▁라', '데', '온', 

In [10]:
print("org: ", org)
print("per: ", per)
print("poh: ", poh)

org:  {'삼성전자', 'Xe', 'AMD', '[IT동아', '인텔', '엔비디아'}
per:  {'트로이', '남시현'}
poh:  {'3D 마크', '지포스 GTX 1650 모바일', 'A770M', 'i5-1240P', '지포스 GTX970', '핸드브레이크', '디아블로 2: 레저렉션', '인텔 아크 시리즈', 'MXM', '인텔 아크 A350M', '토탈워:워해머 II', '인텔 아크 A 시리즈', '아크 컨트롤', 'Xe 미디어', 'MX450M', '인텔 Xe 마이크로 아키텍처', 'PC마크10', 'PC마크 10', '워해머 III', '타임스파이', '갤럭시 북2 프로', '아이리스 Xe', 'XeSS', '인텔 아크 컨트롤', 'GTA V', '레드 데드 리뎀션 II', 'H265', '인텔 코어 i5-1240P', 'GTX 660 2', '다이렉트X 11', '라데온 HD', 'A350M', 'VP9', '라데온 R9 290X', 'GDDR6', '인텔 아이리스 Xe', '지포스 MX450', '갤럭시 북2', 'RTX 3050', 'GTX1650', 'GTX 1650 Ti', 'AV1', '토탈 워 사가', 'R9 290X', '파이어 스트라이크', 'GTX 1060', '라데온 RX 480 4G', '토탈 워 사가:트로이', '트로이', '배틀그라운드', 'Xe', '라이즈 오브 더 툼레이더', '토탈워', '워해머', '워해머 II', '인텔 아크 그래픽 카드', '아크', '라데온 RX 5300M', '지포스 GTX 1660 Ti 모바일', '픽셀 필레이트', '삼국', '레드 데드 리뎀션II', 'GTX 970'}


In [11]:
# diversity값이 클수록, 키워드간의 유사성이 최소화 됨, 문서와 키워드 사이의 유사성은 극대화
mmr_keywords=mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)
print(mmr_keywords)

['인텔', '그래픽카드', '2018년', 'a770m', 'gtx970']


In [12]:
#공백이 있는 후보 키워드들
space_candidate_keywords=set([])
space_candidate_keywords = space_candidate_keywords.union(org)
space_candidate_keywords = space_candidate_keywords.union(per)
space_candidate_keywords = space_candidate_keywords.union(poh)
space_candidate_keywords = space_candidate_keywords.union(mmr_keywords)

candidate_keywords=set([])
for keyword in space_candidate_keywords:
    keyword = keyword.replace(' ', '')
    candidate_keywords.add(keyword)


doc_without_space=doc.replace(' ', '')

count_dict = set([(token, doc_without_space.count(token)) for token in candidate_keywords ])
ranked_words = sorted(count_dict, key=lambda x:x[1], reverse=True)[:20]

ner_sbert_keywords= list([keyword for keyword, freq in ranked_words])

print(ner_sbert_keywords)

['인텔', '그래픽카드', '아크', 'A350M', '인텔아크A350M', 'Xe', '엔비디아', '토탈워', 'AMD', '워해머II', '워해머', 'GTX1650', '레드데드리뎀션II', '토탈워:워해머II', '갤럭시북2프로', '갤럭시북2', 'i5-1240P', '트로이', 'PC마크10', '토탈워사가:트로이']
